In [1]:
import random
random.seed(1000)

# Learning ICMP (PING) Properties

The goal of this notebook is to demonstrate how, in principle, contect-sensitive constraints on top context-free grammars can be learned in a black-box fashion from *patters* of abstract constraints. Here's the definition of the IP/ICMP format from Wikipedia:

<table class="wikitable">
<caption>IPv4 datagram (from <a href="https://en.wikipedia.org/wiki/Ping_(networking_utility)#ICMP_packet">https://en.wikipedia.org/wiki/Ping_(networking_utility)#ICMP_packet</a>)</caption>
<tbody><tr style="background:#dedede">
<th>&#160;
</th>
<th scope="col" style="width:110px;">Bits 0&#8211;7
</th>
<th scope="col" style="width:110px;">Bits 8&#8211;15
</th>
<th scope="col" style="width:110px;">Bits 16&#8211;23
</th>
<th scope="col" style="width:110px;">Bits 24&#8211;31
</th></tr>
<tr style="background:#e8e8ff;">
<th rowspan="5" style="background:#dedede"><a href="/wiki/IPv4_Header" class="mw-redirect" title="IPv4 Header">Header</a><br />(20 bytes)
</th>
<td>Version/IHL
</td>
<td>Type of service (ToS)
</td>
<td colspan="2">Length
</td></tr>
<tr style="background:#e8e8ff;">
<td colspan="2">Identification
</td>
<td colspan="2"><i>flags</i> and <i>offset</i>
</td></tr>
<tr style="background:#e8e8ff;">
<td>Time to live (TTL)
</td>
<td>Protocol
</td>
<td colspan="2">Header checksum
</td></tr>
<tr style="background:#e8e8ff;">
<td colspan="4">Source IP address
</td></tr>
<tr style="background:#e8e8ff;">
<td colspan="4">Destination IP address
</td></tr>
<tr style="background:#ffe8e8;">
<th rowspan="2" style="background:#dedede">ICMP header<br />(8 bytes)
</th>
<td>Type of message
</td>
<td>Code
</td>
<td colspan="2">Checksum
</td></tr>
<tr style="background:#ffe8e8;">
<td colspan="4">Header data
</td></tr>
<tr style="background:#ffe8e8;">
<th style="background:#dedede">ICMP payload<br />(<i>optional</i>)
</th>
<td colspan="4">Payload data
</td></tr></tbody></table>

Generic composition of an ICMP packet:

- IPv4 Header (in blue): protocol set to 1 (ICMP) and Type of Service set to 0.
- ICMP Header (in red):
- Type of ICMP message (8 bits)
  + Code (8 bits)
  + Checksum (16 bits), the 16-bit one's complement of the one's complement sum of the packet. For IPv4, this is calculated from the ICMP message starting with the Type field (the IP header is not included).
  + Header Data (32 bits) field, which in this case (ICMP echo request and replies), will be composed of an identifier (16 bits) and sequence number (16 bits).
  + ICMP Payload: payload for the different kind of answers; can be an arbitrary length, left to implementation detail. However, the packet including IP and ICMP headers must be less than the maximum transmission unit of the network or risk being fragmented.

<table class="wikitable">
    <caption>Echo request ICMP/IPv4 packet</caption>
<tbody><tr>
<th>00</th>
<th>01</th>
<th>02</th>
<th>03</th>
<th>04</th>
<th>05</th>
<th>06</th>
<th>07
</th>
<th>08</th>
<th>09</th>
<th>10</th>
<th>11</th>
<th>12</th>
<th>13</th>
<th>14</th>
<th>15
</th>
<th>16</th>
<th>17</th>
<th>18</th>
<th>19</th>
<th>20</th>
<th>21</th>
<th>22</th>
<th>23
</th>
<th>24</th>
<th>25</th>
<th>26</th>
<th>27</th>
<th>28</th>
<th>29</th>
<th>30</th>
<th>31
</th></tr>
<tr style="text-align:center;">
<td colspan="8">Type = 8(IPv4, ICMP)
</td>
<td colspan="8">Code = 0
</td>
<td colspan="16">Checksum
</td></tr>
<tr style="text-align:center;">
<td colspan="16">Identifier
</td>
<td colspan="16">Sequence Number
</td></tr>
<tr>
<td colspan="32" style="text-align:center;">Payload
</td></tr></tbody></table>

We first define the syntax of ICMP packets embedded into IP packets as a context-free grammar.

In [2]:
icmp_grammar = r"""
    <start>        ::= <ip_packet>
    <ip_packet>    ::= <ip_header> <icmp_packet>
    <ip_header>    ::= <version_ihl> <ToS> <length> <ip_id> <flags_offset> <ttl>
                       <protocol> <ip_checksum> <source_ip> <dest_id>
    <icmp_packet>  ::= <type> <code> <checksum> <icmp_id> <seq> <icmp_payload>
    
    <version_ihl>  ::= <byte>
    <ToS>          ::= <byte>
    <length>       ::= <byte> <byte>
    <ip_id>        ::= <byte> <byte>
    <flags_offset> ::= <byte> <byte>
    <ttl>          ::= <byte>
    <protocol>     ::= <byte>
    <ip_checksum>  ::= <byte> <byte>
    <source_ip>    ::= <byte> <byte> <byte> <byte>
    <dest_id>      ::= <byte> <byte> <byte> <byte>
    
    <type>         ::= <byte>
    <code>         ::= <byte>
    <checksum>     ::= <byte> <byte>
    <icmp_id>      ::= <byte> <byte>
    <seq>          ::= <byte> <byte>
    <icmp_payload> ::= <bytes>
    
    <bytes>        ::= <byte> <bytes> | ""
    <byte>         ::= "\x00" | "\x01" | "\x02" | "\x03" | "\x04" | "\x05"
                     | "\x06" | "\x07" | "\x08" | "\t" | "\n" | "\x0b"
                     | "\x0c" | "\r" | "\x0e" | "\x0f" | "\x10" | "\x11"
                     | "\x12" | "\x13" | "\x14" | "\x15" | "\x16" | "\x17"
                     | "\x18" | "\x19" | "\x1a" | "\x1b" | "\x1c" | "\x1d"
                     | "\x1e" | "\x1f" | " " | "!" | "\"" | "#" | "$" | "%"
                     | "&" | "'" | "(" | ")" | "*" | "+" | "," | "-" | "."
                     | "/" | "0" | "1" | "2" | "3" | "4" | "5" | "6" | "7"
                     | "8" | "9" | ":" | ";" | "<" | "=" | ">" | "?" | "@"
                     | "A" | "B" | "C" | "D" | "E" | "F" | "G" | "H" | "I"
                     | "J" | "K" | "L" | "M" | "N" | "O" | "P" | "Q" | "R"
                     | "S" | "T" | "U" | "V" | "W" | "X" | "Y" | "Z" | "["
                     | "\\" | "]" | "^" | "_" | "`" | "a" | "b" | "c" | "d"
                     | "e" | "f" | "g" | "h" | "i" | "j" | "k" | "l" | "m"
                     | "n" | "o" | "p" | "q" | "r" | "s" | "t" | "u" | "v"
                     | "w" | "x" | "y" | "z" | "{" | "|" | "}" | "~"
                     | "\x7f" | "\x80" | "\x81" | "\x82" | "\x83" | "\x84"
                     | "\x85" | "\x86" | "\x87" | "\x88" | "\x89" | "\x8a"
                     | "\x8b" | "\x8c" | "\x8d" | "\x8e" | "\x8f" | "\x90"
                     | "\x91" | "\x92" | "\x93" | "\x94" | "\x95" | "\x96"
                     | "\x97" | "\x98" | "\x99" | "\x9a" | "\x9b" | "\x9c"
                     | "\x9d" | "\x9e" | "\x9f" | "\xa0" | "\xa1" | "\xa2"
                     | "\xa3" | "\xa4" | "\xa5" | "\xa6" | "\xa7" | "\xa8"
                     | "\xa9" | "\xaa" | "\xab" | "\xac" | "\xad" | "\xae"
                     | "\xaf" | "\xb0" | "\xb1" | "\xb2" | "\xb3" | "\xb4"
                     | "\xb5" | "\xb6" | "\xb7" | "\xb8" | "\xb9" | "\xba"
                     | "\xbb" | "\xbc" | "\xbd" | "\xbe" | "\xbf" | "\xc0"
                     | "\xc1" | "\xc2" | "\xc3" | "\xc4" | "\xc5" | "\xc6"
                     | "\xc7" | "\xc8" | "\xc9" | "\xca" | "\xcb" | "\xcc"
                     | "\xcd" | "\xce" | "\xcf" | "\xd0" | "\xd1" | "\xd2"
                     | "\xd3" | "\xd4" | "\xd5" | "\xd6" | "\xd7" | "\xd8"
                     | "\xd9" | "\xda" | "\xdb" | "\xdc" | "\xdd" | "\xde"
                     | "\xdf" | "\xe0" | "\xe1" | "\xe2" | "\xe3" | "\xe4"
                     | "\xe5" | "\xe6" | "\xe7" | "\xe8" | "\xe9" | "\xea"
                     | "\xeb" | "\xec" | "\xed" | "\xee" | "\xef" | "\xf0"
                     | "\xf1" | "\xf2" | "\xf3" | "\xf4" | "\xf5" | "\xf6"
                     | "\xf7" | "\xf8" | "\xf9" | "\xfa" | "\xfb" | "\xfc"
                     | "\xfd" | "\xfe" | "\xff"
"""

From this grammar, we can already generate random strings that syntactically resemble IP packet-like structures.
To demonstrate this, we build a function leveraging the `scapy` library to parse strings into datastructures.
This function, `ip_packet_from_string`, also sets the source and destination IP addresses to 127.0.0.1 for later demonstration purposes.

In [3]:
import scapy.all as scapy

In [4]:
def ip_packet_from_string(str_packet: str) -> scapy.IP:
    """
    :return: A scapy packet from the given string. Sets source and destination of the packet to 127.0.0.1.
    """
    
    packet = scapy.IP(str(str_packet).encode("latin1"))
    packet[scapy.IP].src = "127.0.0.1"
    packet[scapy.IP].dst = "127.0.0.1"
    return packet

Using the ISLa solver and `ip_packet_from_string`, we generate a random IP packet and inspect its structure.

In [5]:
from isla.solver import ISLaSolver

In [6]:
solver = ISLaSolver(icmp_grammar)
random_ip_string = str(solver.solve())
random_ip_string

'3é]vù6(ÇËÑ|ÒJm9û\x16Á¿EÝI=Äô^\xadî'

In [7]:
packet = ip_packet_from_string(random_ip_string)
packet.show()

###[ IP ]### 
  version   = 3
  ihl       = 3
  tos       = 0xe9
  len       = 23926
  id        = 63798
  flags     = MF
  frag      = 2247
  ttl       = 203
  proto     = 209
  chksum    = 0x7cd2
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \
###[ Raw ]### 
     load      = '\\xddI=\\xc4\\xf4^\\xad\\xee'



This packet does not have a PING layer:

In [8]:
packet.layers()

[scapy.layers.inet.IP, scapy.packet.Raw]

To learn properties of ICMP Echo Requests (aka PING packets), we need some kind of oracle telling us if an IP packet is a correct Echo Request packet.
This can be done most realistically by sending the supposed request, waiting for an answer and asserting that the answer is a ICMP Echo *reply.*
The function `test_ping` accomplishes this.

In [9]:
def test_ping(packet: scapy.IP) -> None:
    TIMEOUT = 1
    scapy.conf.verb = 0
    reply = scapy.sr1(packet, timeout=TIMEOUT)
    return reply is not None and scapy.ICMP in reply.layers()

In [10]:
test_ping(packet)

False

Next, we manually specify a sufficiently strong ISLa constraint to set the baseline for the learning approach.

The following properties need to be formalized for the packet to be valid in all cases (you can later on try to deactivate some of the conjuncts to validate that they are indeed necessary):

- The IP `version_ihl` byte must take the value 45
- The IP `ToS` byte must take the value 0
- The IP `protocol` byte must take the value 1
- The IP `flags_offset` bytes must both take the value 0
- The ICMP `type` byte must take the value 8
- The ICMP `code` byte must take the value 0
- The ICMP `icmp_id` bytes must both take the value 0
- The ICMP `seq` bytes must both take the value 0
- The IP `length` bytes must correctly reflect the length of the entire IP packet, which varies in the length of the variable ICMP payload
- The ICMP checksum `checksum` must be correct (note: the IP checksum `ip_checksum` field seems not to matter, which can also be an interesting result of automatic constraint mining!)

Most constraints can be formalized in "vanilla" ISLa, i.e., with SMT LIB functions and one semantic predicate (`count`).
For the ICMP checksum, however, we have to extend the ISLa language by a suitable semantic predicate.
First, we define a function `compute_internet_checksum` computing that checksum (essentially the 16-bit one's complement of the one's complement sum of the packet).

In [11]:
from typing import Tuple

In [12]:
def compute_internet_checksum(inp_bytes: Tuple[int, ...], length=16) -> int:
    if len(inp_bytes) % 2 != 0:
        inp_bytes = tuple(inp_bytes) + (0x00,)

    assert all(inp_byte <= 0xFF for inp_byte in inp_bytes)
    assert length < 8 or length % 8 == 0

    if length < 8:
        # This is only for testing purposes
        elements = list(inp_bytes)
    else:
        assert length == 16
        elements: List[int] = []
        for i in range(0, len(inp_bytes), 2):
            elements.append((inp_bytes[i] << 8) + inp_bytes[i + 1])

    carry_mask = 2**length
    no_carry_mask = 2**length - 1

    elements_sum = 0
    for element in elements:
        elements_sum += element
        elements_sum = (elements_sum & no_carry_mask) + (
            (elements_sum & carry_mask) >> length
        )

    return elements_sum ^ no_carry_mask

Next, we need a `SemanticPredicate` object for `internet_checksum`.
Each such semantic predicate comes with its own solver that can return `True` or `False` like a normal predicate, but also an *assignment* with a correct solution.
Our solver function `internet_checksum` either returns `True` or a correct assignment, but never `False`.
We do not expect a blackbox constraint inference approach to come up with such complicated implementations itself.
Rather, we assume that we implement re-occurring checksum algorithms and use them in a catalog of constraint patterns.
The Internet checksum, for example, is used in many Internet protocols and possibly other protocols, too, as its idea is not too esoteric.

In [13]:
from typing import Optional

from isla.derivation_tree import DerivationTree
from isla.language import SemPredEvalResult, SemanticPredicate
from isla.type_defs import Grammar

In [14]:
def internet_checksum(
    _: Optional[Grammar], header: DerivationTree, checksum_tree: DerivationTree
) -> SemPredEvalResult:
    if not header.is_complete():
        return SemPredEvalResult(None)

    checksum_nonterminal = checksum_tree.root_nonterminal()

    zero_checksum = (
        checksum_nonterminal,
        [("<byte>", [("\x00", [])]), ("<byte>", [("\x00", [])])],
    )

    header_wo_checksum = header.replace_path(
        header.find_node(checksum_tree),
        DerivationTree.from_parse_tree(zero_checksum),
    )

    header_bytes: Tuple[int] = tuple(str(header_wo_checksum).encode("latin1"))

    checksum_value: str = (
        compute_internet_checksum(header_bytes).to_bytes(2, "big").decode("latin1")
    )

    if checksum_value == str(checksum_tree):
        return SemPredEvalResult(True)

    new_checksum_tree = DerivationTree(
        value=checksum_nonterminal,
        id=checksum_tree.id,
        children=(
            DerivationTree(
                value=checksum_tree.children[0].value,
                id=checksum_tree.children[0].id,
                children=(DerivationTree(checksum_value[0], children=()),),
            ),
            DerivationTree(
                value=checksum_tree.children[1].value,
                id=checksum_tree.children[1].id,
                children=(DerivationTree(checksum_value[1], children=()),),
            ),
        ),
    )

    return SemPredEvalResult({checksum_tree: new_checksum_tree})

Based on the solver function, we define the actual predicate.

In [15]:
INTERNET_CHECKSUM_PREDICATE = SemanticPredicate(
    "internet_checksum", 2, internet_checksum, binds_tree=False, order=100
)

Now, we are set to define the constraints.
Most of them are straightforward; the constraint for the length field, however, deserves some explanation:

```
exists int num_bytes: (
  count(<ip_packet>, "<byte>", num_bytes) and
  str.to.int(num_bytes) < 40 and
  256 * str.to_code(<length>.<byte>[1]) + 
    str.to_code(<length>.<byte>[2]) =
       str.to.int(num_bytes)
)
```

This constraint binds the integer value of the two `<length>` bytes to a numeric variable `num_bytes` using an equation in standard SMT-LIB format (based on the string and integer arithmetic theories) and an existential quantifier.
In the quantifier scope, the solution of this equation is then bound to the number of bytes in the `<ip_packet>` structure using the semantic `count` predicate which is built into ISLa.
The part `str.to.int(num_bytes) < 40` has been added since otherwise, count is currently to slow in producing a solution.
This is an unfortunate implementation issue on which we are currently working.

In [16]:
constraints = r'''
  <version_ihl> = "\u{45}" and
  <ToS> = "\u{0}" and
  <protocol> = "\u{1}" and
  <type> = "\u{8}" and
  <code> = "\u{0}" and
  <flags_offset> = "\u{0}\u{0}" and
  <icmp_id> = "\u{0}\u{0}" and
  <seq> = "\u{0}\u{0}" and
  exists int num_bytes: (
    count(<ip_packet>, "<byte>", num_bytes) and
    str.to.int(num_bytes) < 40 and
    256 * str.to_code(<length>.<byte>[1]) + 
      str.to_code(<length>.<byte>[2]) =
         str.to.int(num_bytes)
  ) and
  internet_checksum(<icmp_packet>, <checksum>)
'''

We construct a solver object, this time with the grammar *and* then semantic constraints, and assert that we produced a valid ICMP Echo Request.

In [17]:
from isla.isla_predicates import COUNT_PREDICATE

In [18]:
solver = ISLaSolver(
    icmp_grammar,
    constraints,
    semantic_predicates={COUNT_PREDICATE, INTERNET_CHECKSUM_PREDICATE},
    max_number_free_instantiations=1,
    max_number_smt_instantiations=10)

In [19]:
ip_string = str(solver.solve())
ip_string

'E\x00\x00 µ\x80\x00\x006\x01~@ý\x9f«x!ô\x08Þ\x08\x00õ\x8b\x00\x00\x00\x00=èÄ\x8b'

In [20]:
packet = ip_packet_from_string(ip_string)
packet.show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 32
  id        = 46464
  flags     = 
  frag      = 0
  ttl       = 54
  proto     = icmp
  chksum    = 0x7e40
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \
###[ ICMP ]### 
     type      = echo-request
     code      = 0
     chksum    = 0xf58b
     id        = 0x0
     seq       = 0x0
     unused    = ''
###[ Raw ]### 
        load      = '=\\xe8ċ'



This packet now is a valid PING request:

In [21]:
packet.layers()

[scapy.layers.inet.IP, scapy.layers.inet.ICMP, scapy.packet.Raw]

In [22]:
test_ping(packet)

True

Now, we would try to come up with this constraint set by combining some atomic *patterns.*
The simplest one has the shape `<?NONTERMINAL> = "<?STRING>"` for two placeholders `<?NONTERMINAL>` and `<?STRING>` for some grammar nonterminal and string literal, respectively.
For this demonstration, we consider an ad-hoc type of patterns:
Functions that, when called without an argument, return either "string" or "nonterminal" to signal what kind of instantiation they expect, and when called with an argument, return a (partially) instantiated pattern.
Some patterns require a grammar for `isla_parse` to work, so we add a grammar parameter.

In [23]:
from typing import Callable, Union
from isla.language import Formula, parse_isla, unparse_isla

In [24]:
Pattern = Callable[[Optional[str], Optional[Grammar]], Union[str, 'Pattern', Formula]]

Below, you find a pattern implementation for the simple "string equation" pattern.

In [25]:
def equation_pattern(str_inst: Optional[str] = None, _ = None) -> Union[str, Pattern]:
    if str_inst is None:
        return "string"
    
    str_inst = "".join([f"\\u{{{hex(ord(c))[2:]}}}" for c in str_inst])
    
    def inner_pattern(nonterminal_inst: Optional[str] = None, _ = None) -> Union[str, Formula]:
        if nonterminal_inst is None:
            return "nonterminal"
        return parse_isla(f'(= {nonterminal_inst} "{str_inst}")')
    
    return inner_pattern

Let us, step by step, reproduce the atomic constraint `<version_ihl> = "\u{45}"`.
Note that this simple equational constraint is syntactic sugar for an ISLa formula with a universal quantifier, so the string representation of the actual result we obtain will look a little differently.

In [26]:
equation_pattern()

'string'

In [27]:
equation_pattern("\x45")()

'nonterminal'

In [28]:
equation_pattern("\x45")("<version_ihl>")

ForallFormula(BoundVariable("version_ihl", "<version_ihl>"), Constant("start", "<start>"), SMTFormula(version_ihl == "E", BoundVariable("version_ihl", "<version_ihl>"), instantiated_variables=OrderedSet({}), substitutions={}))

In [29]:
print(unparse_isla(equation_pattern("\x45")("<version_ihl>")))

forall <version_ihl> version_ihl in start:
  (= version_ihl "E")


And for a two-byte constraint:

In [30]:
print(unparse_isla(equation_pattern("\x00\x00")("<flags_offset>")))

forall <flags_offset> flags_offset in start:
  (= flags_offset "\u{0}\u{0}")


What remains are the patterns for length and Internet checksum.

In [31]:
def length_pattern(nonterminal_inst_1: Optional[str] = None, grammar: Optional[Grammar] = None) -> Union[str, Pattern]:
    if nonterminal_inst_1 is None:
        return "nonterminal"
    
    assert grammar is not None
    
    def inner_pattern_1(nonterminal_inst_2: Optional[str] = None, _ = None) -> Union[str, Pattern]:
        if nonterminal_inst_2 is None:
            return "nonterminal"
    
        def inner_pattern_2(nonterminal_inst_3: Optional[str] = None, _ = None) -> Union[str, Pattern]:
            if nonterminal_inst_3 is None:
                return "nonterminal"
    
            def inner_pattern_3(nonterminal_inst_4: Optional[str] = None, _ = None) -> Union[str, Formula]:
                if nonterminal_inst_4 is None:
                    return "nonterminal"
                
                return parse_isla(f'''
exists int num_bytes: (
  count({nonterminal_inst_1}, "{nonterminal_inst_2}", num_bytes) and
  str.to.int(num_bytes) < 40 and
  256 * str.to_code({nonterminal_inst_3}.{nonterminal_inst_4}[1]) + 
    str.to_code({nonterminal_inst_3}.{nonterminal_inst_4}[2]) =
       str.to.int(num_bytes)
)
''', grammar=grammar, semantic_predicates={COUNT_PREDICATE})
            
            return inner_pattern_3
        
        return inner_pattern_2
    
    return inner_pattern_1

In [32]:
print(unparse_isla(length_pattern("<ip_packet>", icmp_grammar)("<byte>")("<length>")("<byte>")))

forall <ip_packet> ip_packet in start:
  forall <length> length="{<byte> byte}{<byte> byte_0}" in start:
    exists int num_bytes:
      ((count(ip_packet, "<byte>", num_bytes) and
       (< (str.to.int num_bytes) 40)) and
      (= (+ (* 256 (str.to_code byte)) (str.to_code byte_0)) (str.to.int num_bytes)))


In [33]:
def internet_checksum_pattern(nonterminal_inst_1: Optional[str] = None, _ = None) -> Union[str, Pattern]:
    if nonterminal_inst_1 is None:
        return "nonterminal"
    
    def inner_pattern(nonterminal_inst_2: Optional[str] = None, _ = None) -> Union[str, Formula]:
        if nonterminal_inst_2 is None:
            return "nonterminal"
        
        return parse_isla(
            f'internet_checksum({nonterminal_inst_1}, {nonterminal_inst_2})',
            semantic_predicates={INTERNET_CHECKSUM_PREDICATE}
        )
    
    return inner_pattern

In [34]:
print(unparse_isla(internet_checksum_pattern("<icmp_packet>")("<checksum>")))

forall <checksum> checksum in start:
  forall <icmp_packet> icmp_packet in start:
    internet_checksum(icmp_packet, checksum)


All in all, we have a set of three patterns:

In [35]:
patterns = [
    equation_pattern,
    length_pattern,
    internet_checksum_pattern,
]

Assuming for now that we "magically" learned about all the relevant string constants `\u{0}`, `\u{1}`, `\u{8}`, `\u{45}`, and `\u{0}\u{0}`, we still obtain a quite high large search space (even if we only consider actually relevant patterns, i.e., the three in our `patterns` set):

In [36]:
constants = ["\x00", "\x01", "\x08", "\x45", "\x00\x00"]

In [37]:
from typing import List

In [38]:
def instantiate_patterns(patterns: List[Pattern], grammar: Grammar) -> List[Formula]:
    atomic_candidate_constraints = []
    queue = list(patterns)

    while queue:
        pattern = queue.pop()
        if isinstance(pattern, Formula):
            atomic_candidate_constraints.append(pattern)
            continue
        
        try:
            if pattern() == "string":
                for constant in constants:
                    queue.append(pattern(constant, grammar))
            else:
                assert pattern() == "nonterminal"
                for nonterminal in grammar:
                    queue.append(pattern(nonterminal, grammar))
        except SyntaxError:
            continue

    return atomic_candidate_constraints

In [39]:
from isla.language import parse_bnf

In [40]:
instantiations = instantiate_patterns(patterns, parse_bnf(icmp_grammar))

In [41]:
len(instantiations)

644

To come up with the constants, we could capture some valid ICMP Echo Request packets and collect the values they contain (or use scrapy to create some packets).
Ideally, we would somehow only instantiate "suitable" constants; e.g., in an equation, only those found for the nonterminal they are compared agains would be chosen.
Generating with all combinations of n elements of `instantiations`, where the correct n is even unknown, seems not feasible.
What we would do is to take again valid ICMP packets obtained from an arbitrary, trustworthy source and filter those constraints that do not hold for one of these packets.

That way, we would be able to find a conjunction of sensible constraints.
Note that this conjunction could be insufficient for generating *only* valid constraints, e.g., because a required pattern is not present.
Furthermore, some pattern instantiations might hold *coincidentally,* i.e., are not required for valid constraints.
To eliminate these possiblities, we would use a constraint set for generation and check if it produces valid constraints.
Afterward, we could remove constraints and see if the generated results are *still* valid.

So, here's a valid packet generated by scapy:

In [42]:
one_good_packet = scapy.raw(scapy.IP()/scapy.ICMP())
one_good_packet

b'E\x00\x00\x1c\x00\x01\x00\x00@\x01|\xde\x7f\x00\x00\x01\x7f\x00\x00\x01\x08\x00\xf7\xff\x00\x00\x00\x00'

In [43]:
scapy.IP(one_good_packet).show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 28
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = 0x7cde
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \
###[ ICMP ]### 
     type      = echo-request
     code      = 0
     chksum    = 0xf7ff
     id        = 0x0
     seq       = 0x0
     unused    = ''



We can parse this packet using an ISLaSolver object.

In [44]:
parsed_good_input = ISLaSolver(icmp_grammar).parse(one_good_packet.decode("latin1"))

Now, we filter out all instantiations that fail this particular input.

In [45]:
from isla.helpers import Exceptional

In [46]:
candidate_patterns = [
    constraint
    for constraint in instantiations
    if Exceptional.of(
        lambda:
            ISLaSolver(
                icmp_grammar, 
                constraint, 
                semantic_predicates={INTERNET_CHECKSUM_PREDICATE, COUNT_PREDICATE}
            ).check(parsed_good_input) is True
    ).recover(lambda _: False, Exception).get()
]

In [47]:
len(candidate_patterns)

79

In [48]:
import random

In [49]:
print("\n\n".join(map(unparse_isla, random.choices(candidate_patterns, k=20))))

forall <type> type in start:
  (= type ")

forall <seq> seq in start:
  forall <ip_packet> ip_packet in start:
    internet_checksum(ip_packet, seq)

forall <length> length in start:
  forall <langle> langle in start:
    internet_checksum(langle, length)

forall <ip_packet> ip_packet in start:
  forall <langle> langle in start:
    internet_checksum(langle, ip_packet)

forall <flags_offset> flags_offset in start:
  forall <start> start in start:
    internet_checksum(start, flags_offset)

forall <dest_id> dest_id in start:
  forall <langle> langle in start:
    internet_checksum(langle, dest_id)

forall <bytes> bytes in start:
  forall <langle> langle in start:
    internet_checksum(langle, bytes)

forall <langle> langle in start:
  forall <ip_id> ip_id in start:
    internet_checksum(ip_id, langle)

forall <checksum> checksum in start:
  forall <ip_packet> ip_packet in start:
    internet_checksum(ip_packet, checksum)

forall <langle> langle in start:
  forall <seq> seq in start:
   

In this set, there are all the really useful instantiations we need; on the other hand, it also contains "vacuously satisfied" constraints that only hold because the nonterminal the constraint quantifies over is absent in our example input. One such constraint is

```
forall <langle> langle in start:
  forall <ip_id> ip_id in start:
    internet_checksum(ip_id, langle)
```

This is simply true because there is no `<langle>` in `parsed_good_input`.

We can now first check whether the combination of all candidates is sufficiently strong; afterward, we can try to reduce the constraint set to the necessary constraints.

The sketched primitive approach should be improved with a *systematic* solution to the search space explosion.
In the ISLearn tool, some measures are already included.
For example, we restrict the search space of string constants if a constant occurs in an equation with a nonterminal to the constants seen in that nonterminal's contexts in the training set.
Yet, this is not satisfactory.

ICMP is an example where we only get a binary feedback when using constraint candidates for generation:
Either, we have a sufficiently strong constraint set and a PING gets through, or the message is silently dropped.
This lack of incremental feedback, which is not present in scenarios like learning programming language constraints, constitutes a challenge for blackbox learning.
Yet, positive (and potentially also negative) training samples can help to filter out certain candiate instantiations, as we have seen.